# Trabajo práctico nº 1

## Paquetes utilizados y otras configuraciones

In [1]:
# Paquetes empleados

import pandas as pd
# import unicodedata
# import numpy as np
# import matplotlib.pyplot as plt
# import matplotlib.ticker as mtick # para manipular etiquetas de eje
# import seaborn as sns

# from scipy.stats import ttest_ind # prueba t de student (complemento)

In [2]:
# Ajustar tamaño de fuente global de gráficos

plt.rcParams.update({'font.size': 14})  # tamaño base de texto

NameError: name 'plt' is not defined

## Parte I

Levantamos la base de datos resultante en el TP 1, según la cual se unieron las bases de EPH de individuos de los trimestres 1 de 2005 junto con la de 2025.

In [ ]:
base = pd.read_excel(r"C:\Trabajos\Maestria\2 año\Tercer trimestre\Taller de programación\BigDataUBA-Grupo1\BigDataUBA-Grupo1\TP2\basei_unida.xlsx") 

## Consigna 1

In [5]:
base.columns

Index(['CODUSU', 'NRO_HOGAR', 'COMPONENTE', 'H15', 'ANO4', 'TRIMESTRE',
       'REGION', 'PONDERA', 'CH04', 'CH06', 'CH07', 'CH08', 'NIVEL_ED',
       'ESTADO', 'CAT_INAC', 'IPCF', 'CH15', 'PP02H', 'PP02I', 'PP10A', 'T_VI',
       'ITF', 'PP02E', 'PERIODO'],
      dtype='object')

In [ ]:
base['edad2'] = base['CH06']^2 # edad al cuadrado

base['respondieron'] = base['ITF'] > 0 # 1 si respondieron, 0 si no respondieron

